Report section
====

Chosen representation & data preprocessing:
----

1. 
2. 
3. 


Method extensions and implementation(standard and extended)
----
1. 
2. 
3. 
    

Performance (training and validation) results
----
1. 
2. 
3. 
    



In [1]:
import numpy as np
import pandas as pd
import os
import math

In [ ]:
class MyBayesCLF():
    def __init__(self):
        self.__data_Class = {}
        self.__each_Class_Words = {}
        self.__each_Class_Word_Num = {}
        self.__p = {}
        self.__unique_Words = []
        self.__result = []
        self.__training_set_size = 0
        self.__each_Class_Words_Count = {}
        self.stop_Words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

    def fit(self, X, y):
        for [abstract, label] in zip(X, y):
            if label not in self.__data_Class:
                self.__data_Class[label] = 1
            else:
                self.__data_Class[label] += 1
            words = self.__preprocess(abstract.split(" "))
            # words = abstract.split(" ")
            if label not in self.__each_Class_Words:
                self.__each_Class_Words[label] = words
            else:
                self.__each_Class_Words[label] += words
        self.__training_set_size = len(X)
        self.__calculate_Word_count()
        self.__calculate_Unique_Words()
        self.__calculate_Prob_Of_Class()
        self.__count_Each_Class_Words()

    def predict(self, X_test):
        count = 0
        processed_WordList = []
        for index, value in X_test.items():
            words = self.__preprocess(value.split(" "))
            # words = X_test[i].split(" ")

            processed_WordList.append(words)
            for g in words:
                if g not in self.__unique_Words:
                    count += 1
        result = []
        for sample in processed_WordList:
            result.append(self.__predict_Item(sample, count))
        self.__result = result
        return result

    def __predict_Item(self, list_Words, count):
        result = {label: value for label, value in self.__p.items()}
        for word in list_Words:
            for label in result.keys():
                if word in self.__each_Class_Words[label]:
                    result[label] =  math.log((self.__each_Class_Word_Num[label][word]+1) / (self.__each_Class_Words_Count[label] + len(self.__unique_Words) + count), 2)
                else:
                    result[label] += math.log( 1 / (self.__each_Class_Words_Count[label] + len(self.__unique_Words) + count), 2)
        sorted_Result = sorted(result.items(), key = lambda a: a[1], reverse = True)
        # print(sorted_Result)
        return sorted_Result[0][0]

    def __preprocess(self, list_Words):
        result = []
        for word in list_Words:
            # remove stop words
            if word in self.stop_Words:
                continue
            # remove digital words like 1, 2, 3
            if word.isdigit():
                continue
            # remove charactor which is meaningless
            if len(word) == 1:
                continue
            result.append(word)
        # print(result)
        return result

    def results_To_csv(self, fileName):
        df = pd.DataFrame([[id, predict] for [id, predict] in zip(range(1, len(
            self.__result) + 1), self.__result)], columns=["id", "class"]).to_csc(fileName, index=False)

    def __calculate_Prob_Of_Class(self):
        self.__p = {}
        for name in self.__data_Class:
            self.__p[name] = math.log(self.__data_Class[name] / self.__training_set_size, 2)
        # print(self.__p)

    def __calculate_Word_count(self):
        for y in self.__data_Class.keys():
            if y not in self.__each_Class_Word_Num:
                self.__each_Class_Word_Num[y] = {}
            for word in self.__each_Class_Words[y]:
                if word not in self.__each_Class_Word_Num[y]:
                    self.__each_Class_Word_Num[y][word] = 1
                else:
                    self.__each_Class_Word_Num[y][word] += 1
        # print(self.__each_Class_Word_Num)

    def __calculate_Unique_Words(self):
        words = []
        for y in self.__each_Class_Word_Num.keys():
            words += self.__each_Class_Word_Num[y].keys()
        self.__unique_Words = set(words)
        # print(self.__unique_Words)
    def __count_Each_Class_Words(self):
        for label in self.__each_Class_Words:
            self.__each_Class_Words_Count[label] = len(self.__each_Class_Words[label])
        # print(self.__each_Class_Words_Count)

    def __str__(self):
        msg = ""
        msg += f"Class in the training set: {self.__data_Class}\n"
        msg += f"Words in each class: {self.__each_Class_Words}\n"
        return msg

In [1]:
class KFolds:
    def __init__(self, n_splits, shuffle = True, seed = 4321):
        self.seed = seed
        self.shuffle = shuffle
        self.n_splits = n_splits
    
    def split(self, X):
        n_samples = X.shape[0]
        indices = np.arange(n_samples)
        if self.shuffle:
            rstate = np.random.RandomState(self.seed)
            rstate.shuffle(indices)
        for test_mask in self._iter_test_masks(n_samples, indices):
            train_index = indices[np.logical_not(test_mask)]
            test_index = indices[test_mask]
            train_set = X.filter(train_index, axis = 0)
            test_set = X.filter(test_index, axis = 0)
            yield train_set, test_set
        
    def _iter_test_masks(self, n_samples, indices):
        fold_sizes = (n_samples // self.n_splits) * np.ones(self.n_splits, dtype = np.int64)
        fold_sizes[:n_samples % self.n_splits] += 1

        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_indices = indices[start:stop]
            test_mask = np.zeros(n_samples, dtype = bool)
            test_mask[test_indices] = True
            yield test_mask
            current = stop
def calScore(pred_List, real_List):
    count = 0
    for pred, real in zip(pred_List, real_List):
        if pred == real:
            count += 1
    return count / len(real_List)

Cross validation on the training set and find the scores of the model.

In [16]:
training_set = pd.read_csv(os.path.join("data", "trg.csv"))
test_Set = pd.read_csv(os.path.join("data", "tst.csv"))

scores = []
kfold = KFolds(5)
for train,valid in kfold.split(training_set):
    myCLF = MyBayesCLF()
    myCLF.fit(X = train["abstract"], y = train["class"])
    
    result = myCLF.predict(valid["abstract"])
    score = calScore(result, valid["class"])
    scores.append(score)
scores

[0.645, 0.65875, 0.66, 0.625, 0.65]

In [4]:
  
count = {}
for item in result:
    if item in count:
        count[item]+= 1
    else:
        count[item] = 1
count

{'B': 391, 'E': 452, 'V': 55, 'A': 102}

In [5]:
test_Set.insert(1, "class", result)
test_Set.drop(["abstract"], axis=1).to_csv('out.csv', index=False)  